In [2]:
# Python Libraries
import random
import math
import numbers
import platform
import copy
import os
import time

# Importing essential libraries for basic image manipulations.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms
import torchvision.transforms.functional as tF
import torchvision.models as models

In [3]:
%matplotlib inline

# Enable/Disable GPU 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### Create custom dataset

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, test=False):
        img_labels = pd.read_csv(annotations_file, header = 0)
        if test:
            self.img_labels = img_labels.iloc[round(0.9 * img_labels.shape[0]):] 
        else:
            self.img_labels = img_labels.iloc[:round(0.9 * img_labels.shape[0])]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
        return image.float(), label

In [5]:
batch_size = 4

transform = transforms.Compose(
    [transforms.RandomVerticalFlip(p=0.5),
     transforms.RandomHorizontalFlip(p=0.5)])

trainset = CustomImageDataset(annotations_file = './data/annotation_file.csv', 
                              img_dir='./data/champion-classifier', transform=transform, test=False)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=0, shuffle=True)

testset = CustomImageDataset(annotations_file = './data/annotation_file.csv', 
                              img_dir='./data/champion-classifier', transform=None, test=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=0, shuffle=True)

### resNet18

In [6]:
class Champion_Net(nn.Module):
    def __init__(self, num_classes, criterion):
        super(Champion_Net, self).__init__()

        # Implement me
        model_ft = models.resnet18()
        model_ft.fc = nn.Linear(512, num_classes)
        
        self.model = model_ft
        self.criterion = criterion
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inp):
        
        pred = self.model(inp)
        return pred

In [7]:
def train_model(model, dataloaders, optimizer, num_epochs=25):
    
    since = time.time()
    acc_list = []
    model.train() # In training mode

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # backward + optimize only if in training phase
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Epoch information
        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        acc_list.append(epoch_acc)

        print('Training Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return acc_list

In [8]:
def eval_model(model, dataloaders):
    
    since = time.time()
    model.eval() # In training mode

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in dataloaders:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():

            # forward
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    overall_loss = running_loss / len(dataloaders.dataset)
    overall_acc = running_corrects.double() / len(dataloaders.dataset)

    print('Evaluation Loss: {:.4f} Acc: {:.4f}'.format(overall_loss, overall_acc))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return overall_acc

### Training

In [37]:
champion_classifer = Champion_Net(86, nn.CrossEntropyLoss()).to(device)

In [38]:
optimizer_SGD = torch.optim.SGD(champion_classifer.parameters(), lr=0.01, momentum=0.9)

In [43]:
train_model(champion_classifer, trainloader, optimizer_SGD, 8)

Epoch 0/7
----------
Training Loss: 0.5141 Acc: 0.8537
Epoch 1/7
----------
Training Loss: 0.4893 Acc: 0.8577
Epoch 2/7
----------
Training Loss: 0.3591 Acc: 0.8976
Epoch 3/7
----------
Training Loss: 0.3938 Acc: 0.8856
Epoch 4/7
----------
Training Loss: 0.3285 Acc: 0.9082
Epoch 5/7
----------
Training Loss: 0.2435 Acc: 0.9375
Epoch 6/7
----------
Training Loss: 0.2749 Acc: 0.9215
Epoch 7/7
----------
Training Loss: 0.2055 Acc: 0.9428
Training complete in 0m 17s


[tensor(0.8537, device='cuda:0', dtype=torch.float64),
 tensor(0.8577, device='cuda:0', dtype=torch.float64),
 tensor(0.8976, device='cuda:0', dtype=torch.float64),
 tensor(0.8856, device='cuda:0', dtype=torch.float64),
 tensor(0.9082, device='cuda:0', dtype=torch.float64),
 tensor(0.9375, device='cuda:0', dtype=torch.float64),
 tensor(0.9215, device='cuda:0', dtype=torch.float64),
 tensor(0.9428, device='cuda:0', dtype=torch.float64)]

### Validating

In [13]:
eval_model(champion_classifer, testloader)

Evaluation Loss: 5.9401 Acc: 0.2857
Training complete in 0m 1s


tensor(0.2857, dtype=torch.float64)

### Save / Load

In [48]:
torch.save(champion_classifer.state_dict(), './model/champion_classifier_bin0.pt')

In [12]:
champion_classifer = Champion_Net(86, nn.CrossEntropyLoss()).to(device)
champion_classifer.load_state_dict(torch.load('./model/champion_classifier_bin0.pt', 
                                              map_location=device))

<All keys matched successfully>

In [13]:
# champion_classifer.eval()
# pred = champion_classifer(inp.float().to(device))

In [14]:
# pred

In [10]:
device

device(type='cpu')